In [1]:
from huggingface_hub import notebook_login

In [2]:
notebook_login()

In [5]:
from transformers import GPT2Tokenizer, GPTNeoForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import logging
from transformers import TrainingArguments, Trainer

from transformers import RobertaForSequenceClassification, BertForSequenceClassification
from transformers import TFRobertaForSequenceClassification


In [4]:
pt_model = RobertaForSequenceClassification.from_pretrained("/home/vs428/project/n2c2_models/roberta_large_ner_so/roberta_large_ner_so/outputs/roberta_large/checkpoint-384/")


In [6]:
tf_model = TFRobertaForSequenceClassification.from_pretrained("/home/vs428/project/n2c2_models/roberta_large_ner_so/roberta_large_ner_so/outputs/roberta_large/checkpoint-384/", from_pt=True)



2023-01-04 00:33:11.668046: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2023-01-04 00:33:11.668201: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-01-04 00:33:11.669328: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Some weights of the P

In [7]:
tf_model = TFRobertaForSequenceClassification.from_pretrained("/home/vs428/project/n2c2_models/roberta_large_ner_so/tf_model")



All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at /home/vs428/project/n2c2_models/roberta_large_ner_so/tf_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [7]:
pt_model.push_to_hub("n2c2-soap-entailment")

CommitInfo(commit_url='https://huggingface.co/vsocrates/n2c2-soap-entailment/commit/0b6bca418737b10e401688d95a38bba2feb6ac0d', commit_message='Upload RobertaForSequenceClassification', commit_description='', oid='0b6bca418737b10e401688d95a38bba2feb6ac0d', pr_url=None, pr_revision=None, pr_num=None)

In [8]:
tf_model.push_to_hub("n2c2-soap-entailment")

# Fix Labels

In [11]:
from transformers import AutoConfig, AutoModelForSequenceClassification

label2id = {'Not Relevant':3, 'Neither':2, 'Indirect':1, 'Direct':0}
id2label = {v:k for k,v in label2id.items()}
# define model checkpoint - can be the same model that you already have on the hub
model_ckpt = "vsocrates/n2c2-soap-entailment"
# define config
config = AutoConfig.from_pretrained(model_ckpt, label2id=label2id, id2label=id2label)
# load model with config
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, config=config)
# export model
model.save_pretrained("/home/vs428/project/n2c2_models/roberta_large_ner_so/roberta_large_ner_so/upload_relabeled")
# "/home/vs428/project/n2c2_models/roberta_large_ner_so/tf_model"


Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [12]:
model.push_to_hub("n2c2-soap-entailment")

CommitInfo(commit_url='https://huggingface.co/vsocrates/n2c2-soap-entailment/commit/ac58c4b99d855db07d3245e4b5fc4c8963ac0e7a', commit_message='Upload RobertaForSequenceClassification', commit_description='', oid='ac58c4b99d855db07d3245e4b5fc4c8963ac0e7a', pr_url=None, pr_revision=None, pr_num=None)

In [13]:
from transformers import AutoConfig, TFAutoModelForSequenceClassification

label2id = {'Not Relevant':3, 'Neither':2, 'Indirect':1, 'Direct':0}
id2label = {v:k for k,v in label2id.items()}
# define model checkpoint - can be the same model that you already have on the hub
model_ckpt = "vsocrates/n2c2-soap-entailment"
# define config
config = AutoConfig.from_pretrained(model_ckpt, label2id=label2id, id2label=id2label)
# load model with config
model = TFAutoModelForSequenceClassification.from_pretrained(model_ckpt, config=config)
# export model
model.save_pretrained("/home/vs428/project/n2c2_models/roberta_large_ner_so/roberta_large_ner_so/tf_upload_relabeled")
# "/home/vs428/project/n2c2_models/roberta_large_ner_so/tf_model"


Downloading:   0%|          | 0.00/956 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at vsocrates/n2c2-soap-entailment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [14]:
model.push_to_hub("n2c2-soap-entailment")

# Also Upload Tokenizer

In [8]:
from functools import partial


In [9]:

import pandas as pd
import numpy as np
import spacy

import shap

In [10]:
import pyarrow

In [11]:
pyarrow.__version__

'7.0.0'

In [12]:

import transformers
from transformers import AutoTokenizer, AutoModel, AutoModelForMaskedLM, AutoModelForSequenceClassification
from transformers import GPT2Tokenizer, GPTNeoForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import logging
from transformers import TrainingArguments, Trainer

from transformers import RobertaForSequenceClassification, BertForSequenceClassification


In [13]:
from datasets import load_dataset
from datasets import Value, ClassLabel, Features, DatasetDict
from datasets import load_metric
import evaluate

In [14]:
from omegaconf import DictConfig, OmegaConf
import hydra

In [15]:
from preprocessing.cleaning_utils import *
from train_utils.metrics import *
from train_utils.plot_utils import *
from train_utils.custom_trainer import *


# Load in Models

In [16]:
base_model_path = "/home/vs428/project/models/RoBERTa-large-PM-M3-Voc/RoBERTa-large-PM-M3-Voc-hf"
model_path = "/home/vs428/project/n2c2_models/roberta_large_ner_so/roberta_large_ner_so/outputs/roberta_large/checkpoint-384/"

In [24]:
tokenizer = transformers.AutoTokenizer.from_pretrained(base_model_path, use_fast=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [25]:
tokenizer

PreTrainedTokenizerFast(name_or_path='/home/vs428/project/models/RoBERTa-large-PM-M3-Voc/RoBERTa-large-PM-M3-Voc-hf', vocab_size=50000, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)})

# Read in Dev Data

In [26]:
data_path = "/home/vs428/project/n2c2/2022/Data/dev_so.csv"
spacy_assessment = "/home/vs428/project/n2c2_spacy_models/assessment_model_v2/model-best"
spacy_plan = "/home/vs428/project/n2c2_spacy_models/plan_subsection_model_v3/model-best"
spacy_so = "/home/vs428/project/n2c2_spacy_models/so_model/model-best"


In [27]:
dev = pd.read_csv(data_path)

In [28]:
dev

,ROW ID,HADM ID,Assessment,Plan Subsection,Relation,HADM_ID,S,O
0,576487,112508,A 60 year old woman with recurrent ALL with CN...,"CNS VRE: S/P Omaya removal, on linezolid for ...",Direct,112508,NGT placed/TF started Femoral line changed o...,Last dose of Antibiotics: Meropenem - [**2176...
1,576487,112508,A 60 year old woman with recurrent ALL with CN...,LEUKOCYTOSIS: GCSF vs culture negative bacte...,Direct,112508,NGT placed/TF started Femoral line changed o...,Last dose of Antibiotics: Meropenem - [**2176...
2,576487,112508,A 60 year old woman with recurrent ALL with CN...,ALL: Currently day +43 s/p hyper-CVAD. She i...,Indirect,112508,NGT placed/TF started Femoral line changed o...,Last dose of Antibiotics: Meropenem - [**2176...
3,576487,112508,A 60 year old woman with recurrent ALL with CN...,DRUG RASH: Appears improved today. Evidence ...,Indirect,112508,NGT placed/TF started Femoral line changed o...,Last dose of Antibiotics: Meropenem - [**2176...
4,576487,112508,A 60 year old woman with recurrent ALL with CN...,ELEVATED LIVER ENZYMES: Consistent with shock...,Neither,112508,NGT placed/TF started Femoral line changed o...,Last dose of Antibiotics: Meropenem - [**2176...
...,...,...,...,...,...,...,...,...
592,679058,187377,"This is a 67 year old male h/o HTN, systolic H...",# Hct Drop: Likely represents GI bleed though ...,Neither,187377,Chest Pain/Multivessel CAD [**4-4**] CP at [*...,Last dose of Antibiotics: Infusions: Nitrogl...
593,679058,187377,"This is a 67 year old male h/o HTN, systolic H...",# HTN: BP well controlled at this time. Trying...,Indirect,187377,Chest Pain/Multivessel CAD [**4-4**] CP at [*...,Last dose of Antibiotics: Infusions: Nitrogl...
594,679058,187377,"This is a 67 year old male h/o HTN, systolic H...",# Hyperlipdemia: cont statin,Neither,187377,Chest Pain/Multivessel CAD [**4-4**] CP at [*...,Last dose of Antibiotics: Infusions: Nitrogl...
595,679058,187377,"This is a 67 year old male h/o HTN, systolic H...",# Tobacco Abuse: Deferrred nicotine patch at t...,Neither,187377,Chest Pain/Multivessel CAD [**4-4**] CP at [*...,Last dose of Antibiotics: Infusions: Nitrogl...


In [29]:

# create hf Dataset
classes = ['Not Relevant', 'Neither', 'Indirect', 'Direct']

# instead we will use the raw text for now
features = Features({
    'ROW ID':Value("int64"),
    'HADM ID':Value("int64"),
    'Assessment':Value("string"),
    'Plan Subsection':Value("string"),
    "Relation":Value("string"),
    "S":Value("string"),
    "O":Value("string")        

}) 

dataset = load_dataset("csv", data_files={
                            # "train":cfg.data.n2c2_data_dir + "train_so.csv",
                            "valid":data_path,
                            # "train":cfg.data.n2c2_data_dir + "train.csv",
                            # "valid":cfg.data.n2c2_data_dir + "dev.csv",

                        },
                       features=features)

# create encoded class labels and rename
label2id = {'Not Relevant':3, 'Neither':2, 'Indirect':1, 'Direct':0}
id2label = {v:k for k,v in label2id.items()}

dataset = dataset.class_encode_column("Relation")
dataset = dataset.align_labels_with_mapping(label2id, "Relation")
dataset = dataset.rename_column("Relation", "label")

# dataset['valid'] = dataset['valid'].shard(num_shards=20, index=0)

# dataset = dataset.map(partial(add_SO_sections))
# print("AFTER TRAIN _SO sections")
print(dataset['valid'][0])

spacy.require_gpu()
nlp_assessment = spacy.load(spacy_assessment, exclude="parser")
nlp_plan = spacy.load(spacy_plan, exclude="parser")

dataset['valid'] = dataset['valid'].map(partial(add_ner_assessment, nlp=nlp_assessment))
dataset['valid'] = dataset['valid'].map(partial(add_ner_plan, nlp=nlp_plan))        

# we ASSUME that the ner labels we want are lowercase, UNLIKE the standard ones in the model
spans = [x for x in nlp_plan.get_pipe("ner").labels if x.islower()] + [x for x in nlp_assessment.get_pipe("ner").labels if x.islower()]

tokens = []
for span in spans:
    tokens.append("<" + span + ">")
    tokens.append("</" + span + ">")            

# add the span tags to the vocab
_ = tokenizer.add_tokens(tokens)

Using custom data configuration default-86e8ba7bd129054c
Found cached dataset csv (/home/vs428/.cache/huggingface/datasets/csv/default-86e8ba7bd129054c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/vs428/.cache/huggingface/datasets/csv/default-86e8ba7bd129054c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-e3d1802179efc530.arrow
Loading cached processed dataset at /home/vs428/.cache/huggingface/datasets/csv/default-86e8ba7bd129054c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-0c3ea877a77644f2.arrow
[W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.4.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate


{'ROW ID': 576487, 'HADM ID': 112508, 'Assessment': 'A 60 year old woman with recurrent ALL with CNS involvement s/p Omaya\n   removal due to VRE contamination & SDH evacuation.  She is now doing\n   well and awake s/p extubation, afebrile and her WBC count is trending\n   downward.', 'Plan Subsection': 'CNS VRE:  S/P Omaya removal, on linezolid for greater CNS penetration\n   despite risk of CNS penetration. No TEE performed.\n   - continue linezolid (D1=[**4-16**]), course to be determined by TEE (4 vs 8\n   weeks) gentamicin D/C\nd per ID\n   - f/u culture data and sensitivities\n   - TEE deferred at this time.\n   - f/u tigacycline sensitivities for long-term therapy', 'label': 0, 'S': ' NGT placed/TF started  Femoral line changed over wire by IR  c diff x 2 negative, 3rd pending  d/c\'ed gentamicin per ID Sulfa (Sulfonamides)  Rash;  Penicillins  Rash;  Latex  Hives; Wheezing  Red Dye  Headache; "thra  Darvon (Oral) (Propoxyphene Hcl)  Nausea/Vomiting  Percodan (Oral) (Oxycodone H

  0%|          | 0/597 [00:00<?, ?ex/s]

  0%|          | 0/597 [00:00<?, ?ex/s]

In [30]:
tokenizer.push_to_hub("n2c2-soap-entailment")

CommitInfo(commit_url='https://huggingface.co/vsocrates/n2c2-soap-entailment/commit/59c09d100ad030f2e542a40c145801a6bb544960', commit_message='Upload tokenizer', commit_description='', oid='59c09d100ad030f2e542a40c145801a6bb544960', pr_url=None, pr_revision=None, pr_num=None)